In [2]:
    from importnb import Notebook, Main, reload, parameterize, Parameterize
    from importnb.helpers import IMPORTED, INTERACTIVE, CLI, MAIN
    import sys
    from pytest import fixture, raises, mark
    import json, linecache, inspect, ast

In [8]:
    try: from IPython import get_ipython
    except: get_ipython = lambda: None

In [39]:
    def test_basic():
        with Notebook(): import foobar
        assert isinstance(foobar.__loader__, Notebook)
        del foobar

In [40]:
    @fixture
    def module():
        sys.path_importer_cache.clear()
        with Notebook(): import foobar
        yield foobar
        del foobar

In [41]:
    @fixture
    def package():
        with Notebook(): 
            import foobaz.foobar
        yield foobaz.foobar
        del foobaz

In [42]:
    def test_package(package, module):
        assert not module.__package__
        assert package.__package__

In [43]:
    def test_main(module): 
        new = Main.load(module.__file__)
        assert new is not module, """A copy of the module was not made"""
        assert new.__name__ == '__main__'
        assert __import__(new.__name__) != new, """The loader replaced main."""

In [44]:
    def test_reload(module): 
        """delete a method from the module and see if it is recovered"""
        del module._repr_markdown_
        
        """The contextmanager is required."""
        with Notebook():
            reload(module)
        assert hasattr(module, '_repr_markdown_')

In [45]:
    def test_docstrings(module):
        assert module.__doc__
        assert module.function_with_markdown_docstring.__doc__
        assert module.function_with_python_docstring.__doc__

In [46]:
    def test_docstring_opts(module):
        with Notebook(_markdown_docstring=False):
            reload(module)
        assert module.__doc__
        assert not module.function_with_markdown_docstring.__doc__
        assert module.function_with_python_docstring.__doc__

In [47]:
    def test_from_file(module):
        new = Notebook.load(module.__file__)
        assert module is not new

In [48]:
    def test_module_source(module): 
        with raises(json.JSONDecodeError):
            json.loads(''.join(linecache.cache[module.__file__][2]))
        assert inspect.getsource(module).strip() == ''.join(linecache.cache[module.__file__][2]).strip()

In [49]:
    def test_object_source(module): 
        assert ast.parse(inspect.getsource(module.function_with_markdown_docstring)), """The source is invalid"""

In [50]:
    @mark.skipif(not get_ipython(), reason="""Not IPython.""")
    def test_with_shell(module): 
        with Notebook(): reload(module)
        assert not module.__loader__._shell
        with Notebook(_shell=True): reload(module)
        assert module.__loader__._shell

In [51]:
    def test_python_file():
        import foobar as module
        assert reload(module).__file__.endswith('.py'), """Python didn't take precedent."""
        with Notebook(): assert reload(module).__file__.endswith('.ipynb')
        assert reload(module).__file__.endswith('.py')
        with Notebook(): assert reload(module).__file__.endswith('.ipynb')

In [52]:
    @mark.skipif(not get_ipython(), reason="""Not IPython.""")
    def test_cli(module): 
        !ipython -m $module.__name__

In [53]:
    def test_parameterize(module):
        f = parameterize(module)
        assert 'a_parameter' in f.__signature__.parameters
        assert 'not_a_parameter' not in f.__signature__.parameters
        assert isinstance(f(), type(module))

In [54]:
    def test_minified_json(module): 
        with open(module.__file__) as f, open('foobarmin.ipynb', 'w') as o: 
            json.dump(json.load(f), o, indent=None)
            
        with Notebook():
            import foobarmin 
            
        assert inspect.getsource(foobarmin.function_with_markdown_docstring)
            
        with open(foobarmin.__file__) as file:
            assert json.load(file)

In [55]:
    def test_fuzzy_finder(): 
        import __bar
        assert __bar.__name__ == 'foobar'
        del __bar

In [56]:
    def test_remote():  
        ...


In [57]:
    def test_helpers():
        assert IMPORTED
        assert not any({CLI, INTERACTIVE, MAIN})

In [58]:
    if __name__ == '__main__':
        !ipython -m pytest -- -s -p no:pytest-importnb tests.ipynb

============================= test session starts =============================
platform win32 -- Python 3.6.5, pytest-3.5.1, py-1.5.3, pluggy-0.6.0 -- C:\Anaconda3\python.exe
cachedir: ..\..\..\.pytest_cache
rootdir: C:\Users\deathbeds\importnb, inifile: tox.ini
plugins: testmon-0.9.12, remotedata-0.2.1, openfiles-0.3.0, doctestplus-0.1.3, arraydiff-0.2, hypothesis-3.66.16
collecting ... 
======================== no tests ran in 0.02 seconds =========================
---------------------------------------------------------------------------
SystemExit                                Traceback (most recent call last)
C:\Anaconda3\lib\runpy.py in run_module(mod_name, init_globals, run_name, alter_sys)
    203         run_name = mod_name
    204     if alter_sys:
--> 205         return _run_module_code(code, init_globals, run_name, mod_spec)
    206     else:
    207         # Leave the sys module alone

C:\Anaconda3\lib\runpy.py in _run_module_code(code, init_globals, mod_name, mod_spec

ERROR: not found: C:\Users\deathbeds\importnb\src\importnb\tests\tests.ipynb
(no name 'C:\\Users\\deathbeds\\importnb\\src\\importnb\\tests\\tests.ipynb' in any of [])

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2634: UserWarning: Unknown failure executing module: <pytest>
  warn('Unknown failure executing module: <%s>' % mod_name)
